### Try to build a an agent that will work with client to compile and execute a sql order
 - bonus: build a tool to email the order details (including price) when done

In [1]:
# open ai authentication

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
import dbio

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
# email util

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from langchain.agents import tool

def get_mail_server(server='smtp.gmail.com', port=587):
    '''Allocate mail server on port'''
    mailServer = smtplib.SMTP(server, port)
    test = mailServer.starttls()  # start TLS
    assert test[1].lower().find(b'ready') != -1
    return mailServer

def format_message(message, sender, receiver, subject):
    '''
    Format arguments into MIMEMultipart object
    '''

    # message
    msg = MIMEMultipart()
    msg['From'] = sender
    msg['To'] = ', '.join(receiver)
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'html'))

    return msg

@tool
def send_message(order_summary, email='user@user.com'):
    """Returns a tuple of status and the email to the user. \
    Use this for any questions requesting an email summary of \
    the order. Do not call this function until the user has \
    provided an email address. The input should the the summary \
    email and the email provided by the user. \
    The consruction of the email message should occur outside \
    outside this function."""
    
    sender = 'OrderBotPizza.com'
    subject = 'Your OrderBot Pizza Order'
    verbose = True

    msg = format_message(order_summary, sender, [email], subject)
    '''
    if verbose:
        print(f'sending mail from {sender} to {receiver}')
    with get_mail_server() as mailServer:
        mailServer.sendmail(sender, receiver, msg.as_string())
    '''
    print(msg)
    return msg, 'mail sent successfully'

In [ ]:
send_message('hello time')

#enabling gmail will require 2-factor authentication with google

# Instantiate the components separately

In [ ]:
# langchain
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor

In [ ]:
# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, verbose=True)

# database
db = SQLDatabase.from_uri("sqlite:///./pizza_v2.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [ ]:
agent_executor.run('describe the pizzas table')

In [ ]:
from langchain.agents import initialize_agent

In [ ]:
initialize_agent?

# try with general chat and sql tool

 - following the example [here](https://python.langchain.com/en/latest/modules/agents/agents/examples/chat_conversation_agent.html) of an agent that can call tools and chat with the usert too
 - Note there is no prompt specific setting the context for the OrderBot

In [3]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [4]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [5]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, verbose=True)

In [6]:
db = SQLDatabase.from_uri("sqlite:///./pizza_v2.db")
from langchain.chains import SQLDatabaseSequentialChain
seq_chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)

In [7]:
tools = [
    Tool(
        name = "menu search",
        func=seq_chain.run,
        description="""useful for when you need to answer questions about menu items the user wants to order.
        Search your chat history before deciding you need to use this."""
    ),
        Tool(
        name = "send email",
        func=send_message,
        description="""Used when sending the use the summary email of the order. Call this only when the user \
        has finished entering their order and provided an email address."""
    ),
]

In [8]:
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
                               verbose=True, memory=memory)

#agent_chain.agent.llm_chain.prompt = ChatPromptTemplate goes here, put details in system role if possible

"""
#"""
# access chat memory with agent_chain.memory

In [9]:
agent_chain.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, A

In [ ]:
agent_chain.run(input="hi, i am Tim and my email is t@gmail.com")

In [ ]:
agent_chain.agent.llm_chain.prompt # how inject the orderBot prompt with specific instrunctions?

In [ ]:
agent_chain.run(input="what is my name and email")

In [ ]:
agent_chain.run('I would like to order a large pizz and bottled water')

In [ ]:
agent_chain.run('What sizes of bottled water do you have?')

In [ ]:
#agent_chain.run('I will take the bottled water.  What is you most popular pizza?')
agent_chain.run('Add the large bottled water to my order.')

In [ ]:
agent_chain.run('Add the bottled water to my order.')

In [ ]:
agent_chain.run(input="""I would like the large pepperoni \
pizza. Please include both sausage and extra cheese as additional \
toppings.
""")

In [ ]:
agent_chain.run(input="Please repeat my order to make sure it is right.")

In [ ]:
agent_chain.run(input="that is it. please email me my receipt")

Try again with custom bot following example here: https://python.langchain.com/en/latest/modules/agents/agents/custom_llm_agent.html

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish

from langchain.sql_database import SQLDatabase
from langchain.chains import SQLDatabaseSequentialChain

from langchain.memory import ConversationBufferMemory
#from langchain.agents import initialize_agent # alternative to AgentExecutor?
#from langchain.agents import AgentType

import re

In [ ]:
# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, verbose=True)

# memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# db
db = SQLDatabase.from_uri("sqlite:///./pizza_v2.db")
seq_chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)

# tools
tools = [
    Tool(
        name = "menu search",
        func=seq_chain.run,
        description="useful for when you need to answer questions about menu items the user wants to order"
    ),
]

In [ ]:
# prompt template

# Set up the base template
template_with_history = """You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \

You have access to the following tools: \

{tools} \

Use the following format: \

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [ ]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [ ]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [ ]:
# Output parser
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

In [ ]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [ ]:
agent_executor.run("How many pizzas do you have?")